In [6]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
from glob import glob
from PIL import Image

# Get Manga Chapters

In [7]:
def get_manga_chapters(name):
    url = f"https://tortuga-ceviri.com/manga/{name}/ajax/chapters/"
    
    req = requests.post(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    listing_chapters = soup.find('ul', {'class':'main version-chap no-volumn'})
    
    chapters = [i.find('a')['href'] for i in listing_chapters.find_all('li',{"class":"wp-manga-chapter"})]
    return chapters

In [8]:
berserk_chapters = get_manga_chapters("berserk")

In [9]:
berserk_chapters

['https://tortuga-ceviri.com/manga/berserk/berserk-371/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-370_1/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-369/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-368/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-367/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-366/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-365/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-364/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-363/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-362/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-361/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-360/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-359/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-358/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-357/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-356b/',
 'https://tortuga-ceviri.com/manga/berserk/berserk-356a/',
 'https://

# Get Manga Page

In [10]:
chapter_371 = berserk_chapters[0]

In [11]:
def get_manga_pages(chapter):
    url = chapter
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    reading_content = soup.find('div', {'class':'reading-content'})
    pages = reading_content.find_all('div', {'class':'page-break no-gaps'})
    
    manga = [page.find('img')['src'].strip() for page in pages]
    return manga

In [12]:
manga_pages_list = get_manga_pages(chapter_371)

# Download Data

In [13]:
def resize_by_percentage(image, percentage = 1.1789600967351874):
    width, height = image.size
    resized_dimensions = (int(width * percentage), int(height * percentage))
    resized = image.resize((827,1200))
    return resized

In [14]:
def remove_dir_file(dir_path):
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            os.rmdir(file_path)

In [15]:
def download_data(manga):
    for i, url in enumerate(manga):
        response = requests.get(url)
        file_name = f"image{i}.jpg"  # her resim için benzersiz bir dosya adı oluştur
        folder_path = "images"  # resimleri indireceğiniz klasörün yolu
        os.makedirs(folder_path, exist_ok=True)  # klasörü oluştur (eğer yoksa)
        file_path = os.path.join(folder_path, file_name)  # resmin dosya yolu
        with open(file_path, "wb") as f:
            f.write(response.content)

In [16]:
def get_manga_pdf(manga_chapter):
    page_list = glob('./images/*.jpg')
    folder_path = "images"  # resimleri indireceğiniz klasörün yolu
    os.makedirs("pdf", exist_ok=True)

    rgb_images = []
    for page in page_list:
        image = Image.open(page)    
        im_rgb = image.convert('RGB')
        
        width, height = im_rgb.size
        if width > height:
            left_half = im_rgb.crop((0, 0, width // 2, height))
            right_half = im_rgb.crop((width // 2, 0, width, height))
            rgb_images.extend([resize_by_percentage(left_half), resize_by_percentage(right_half)])
        else:
            rgb_images.append(resize_by_percentage(im_rgb))
            
    rgb_images[0].save(f'./pdf/{manga_chapter}.pdf', save_all=True, append_images=rgb_images[1:])

    remove_dir_file("images")
    os.rmdir("images")

In [17]:
download_data(manga_pages_list)

In [18]:
get_manga_pdf("berserk_371")#direkt manga isminden nasıl bulunur onu eklicem